# 五章 誤差逆伝播法
- 前章では損失関数の勾配は数値微分で求めたが、simpleではあるものの計算に時間がかかる
- 誤差逆伝播法は効率よく購買を計算する方法

## 計算グラフ
- 計算グラフとは計算過程をグラフで表現したもの(p124)
- ノードと矢印があり、
    - ノード: 演算を表す
    - 矢印: 現在の金額
- 計算の解き方
    1. 計算グラフを構築する
    2. 計算グラフ上で計算を左から右へすすめる ... 順伝播
        - 逆伝播はこの計算を右から進める
- why 計算グラフを導入するのか
    - 全体が複雑な計算であっても、局所的な計算に分けることで問題を単純化できる
    - 途中の計算を保存できる
    - ! 微分を効率よく計算できる
- p128: リンゴの値段が微小に値上がりした場合の支払額への影響(δL/δx)を計算する
    - どうやるかは後述
    
## 連鎖率
- 計算グラフの逆伝播
    - 上流から伝達された値に、ノードの局所的な微分を蒸散して伝達していく
- 合成関数とは
    - 複数の関数によって構成される関数のこと
        - ex: z = (x+y)** <=> z = t**2, t = x + y
- 連鎖律の原理
    - ある関数が合成関数で表される場合、その合成関数の微分は合成関数を構成するそれぞれの関数の微分の席で表すことができる
    - ex: δz/δx(xに関するzの微分) = δz/δｔ　* δt/δx
    - ex: δz/δy(yに関するzの微分) = δz/δｔ　* δt/δy
    - ex: δz/δt = 2t, δt/δx = 1、よってδz/δx = 2t = 2(x+y)
- 連鎖率を用いて、計算グラフを逆伝播にたどり微分を求めることが出来た  
  
## 逆伝播
- 演算子毎の逆伝播を見ていく
- 加算ノードの逆伝播
    - 加算なので、z = x + y
    - よって、δz/δx = 1
- 乗算ノードの逆伝播
    - 乗算なので、z = xy
    - よって、δz/δx = y, δz/δy = x
- [ ] 確かに計算グラフを手で問いて確かめると納得は行く
    - これは体に染み込ませて理解させるパターン

In [6]:
# 実装する

## 乗算layer ... 計算グラフのノードをlayerと今後呼ぶ
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    # 順伝播
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out

    def backword(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy
    

apple = 100
apple_num = 2
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forwrod
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)
# print(price) # 220

# backword
dprice = 1 # priceを1円としたとき
dapple_price, dtax = mul_tax_layer.backword(dprice)
dapple, dapple_num = mul_apple_layer.backword(dapple_price)
# print(dapple, dapple_num, dtax) # 2.2 110.00000000000001 200

## 加算layer
class AddLayer:
    def __init__(self):
        pass # 何もしない
    
    def forward(self, x, y):
        return x + y
    
    def backword(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy
    
orange = 150
orange_num = 3

mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()

orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

dprice = 1
dall_price, dtax = mul_tax_layer.backword(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backword(dall_price)
dapple, dapple_num = mul_apple_layer.backword(dapple_price)
dorange, dorange_num = mul_orange_layer.backword(dorange_price)

# print(price) # 715
# print(dall_price, dtax, dapple_price, dorange_price, dapple, dapple_num, dorange, dorange_num)
# 1.1 650 1.1 1.1 2.2 110 3.3 165.0

'''
## 活性化関数レイヤ(ノード, ニューロン)の実装
- 計算グラフのアイデアを元にNNのレイヤをクラスとして実装する
- 活性化関数としてReLUとsigmoidを考える
- ReLU
    - x -> (relu) -> y ... という計算グラフを考える
    - x > 0 の時 y = xなので、δy/δｘ = 1, そうでない時は０
'''

class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0) # True/Falseからなるndarray
        out = x.copy()
        out[self.mask] = 0 # 入力地が0以下のものは全て0に置き換えて(maskして)返す
        
        return

    def backword(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

class Sigmoid:
    def __init__(self):
        self.out = None
        pass
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        pass
    
    def backword(self, dout):
#         step1 = dout * (-1) * self.out ** 2 # y = 1/tにおいて δy/δt = -(1/t**2) = -y**2
#         step2 = step1 * 1      # 加算ノードのbackwordは 1をかけるだけ
#         step3 = step2 * np.exp(-self.x) # (e**x)' = e**x
#         step4 = step3 * (-1) # 乗算ノードのbackword
        # 整理すると、step4 = dx = dout * self.out ** 2 * np.exp(-self.x)
        # 今、self.out ** 2 * np.exp(-self.x)を変形すると、out * (1 - out)になる
        # よって
        dx = dout * self.out * (1.0 - self.out) # 結果、self.xを保持しなくて良くなった
        return dx
    

'''
## Affine / Softmaxレイヤの実装
- Affineレイヤ
    - NNの順伝播で行なう行列の内積は、幾何学でアフィン変換と呼ばれる
    - ようはnp.dotを行なうレイヤーを考える
- ココからは入力値はスカラではなく、行列だと考える
    - [ ] p149, p150を手で問いて納得するしか無い
- バッチ版Affineレイヤ
    - 各X, W, Bを複数レコード蓄えられるようにする
- [ ] numpyにおけるaxisがよくわかっていない
    - 0番目の軸(データを単位とした軸)とは
- p153上部の図がわかりやすい
- 補足: NNの処理には推論と学習の2つがある
    - 推論では、最後の出力層において、合算(affine)のあと、わざわざsoftmaxなどで正規化する必要がない
        - なぜなら最大値にだけ興味があるので、別に序列を変更しないsoftmaxをかませる必要がない
        - この正規化する前の出力状態をスコアと呼ぶ
    - 一方、学習においてはsoftmaxが必要
        - [ ] why
            - 学習においては最終的なoutputがスコアではなく、（スコアからだした推論の結果と正解との誤差を示す）損失関数の値であるため
            - さて、正解データは基本的にone_hot_labelであるため、0/1で表されている
            - なので、softmaxで最終的なoutputを(スコアとは違って)0-1に収めた形にしないと正しく誤差を算出できない
                - 再掲: 損失関数では重に二乗和誤差(0.5 * ∑(yk - tk)^2)や、交差エントロピー誤差(- ∑(tk * log(yk)) )が用いられる
- 学習の最後で損失関数を計算する際、適切な組み合わせがある
    - 分類問題: 出力層にsoftmaxを選び出力したyと正解tを元に、交差エントロピー誤差を損失関数として選択する
    - 回帰問題: 出力層に恒等関数を選び出力したyと正解tを元に、二乗和誤差を損失関数として選択する
- こうすると、(計算の詳細を省くが)損失関数に対する逆伝播が(t-y)、即ち「教師ラベル - 出力」というキレイな形に落ち着く
    - というのも、そうなるように狙って二乗和誤差と、交差エントロピー誤差が設計されている
    - [ ] ここらへん統計と大分絡んできそう。
- 
'''

class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backword(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dW = no.dot(self.x.T, dout)
        self.db = no.sum(dout, axis=0) # これがわからない
        return dx
        
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx
    
'''
## 誤差逆伝播法の実装
- 誤差逆伝播法に対応したNNの実装
    - 4章のTwoLayerNetとほぼ同じような実装
- [ ] SoftmaxWithLossのbackwardで最後に割っているのはなんでだ
- [ ] TwoLayerNetのaccracyで最後に割っているのはなんでだ
- 数値微分と誤差逆伝播法の使い分け
    - 通常は誤差逆伝播法を利用し、
    - 数値微分でその実装の正しさを確認する
    - 誤差逆伝播法のデメリットはその複雑さ故に実装のミスを誘発しやすこと
- 勾配確認 ... 両者で求めた勾配の結果がほとんど近いことを確かめる作業
'''


IndexError: arrays used as indices must be of integer (or boolean) type

In [8]:
import sys, os
sys.path.append('./deep-learning-from-scratch')
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    # std: 標準偏差
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['b2'] = np.zeros(output_size)
        
        # レイヤの作成
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(x, t)
    
    def accracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis = 1)
        if t.ndim != 1: t = np.argmax(t, axis=1)
            
        accracy = np.sum(y == t) / float(x.shape[0])
        return accracy
                                                 
    # 数値微分. simpleだけど遅い
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W,self.params['W1'])
        grads['W2'] = numerical_gradient(loss_W,self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W,self.params['b1'])
        grads['b2'] = numerical_gradient(loss_W,self.params['b2'])
        
        return grads
    
    # 逆伝播を用いた勾配の求め方
    def gradient(self, x, t):
        # forward
        self.loss(x, t)
        
        # backword
        dout = 1
        dout = self.lastLayer.backward(dout)
        layers = list(self.layers.values())
        layers.reverse()
        # 各layerでbackwardすることで、dW, dbを蓄える
        for layer in layers:
            dout = layer.backward(dout)
            
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['W2'] = self.layers['Affine2'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['b2'] = self.layers['Affine2'].db
        
        return grads
    

# 勾配確認

from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

nw = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# 確認だけなので4つあれば十分
x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = nw.numerical_gradient(x_batch, t_batch)
grad_backpop = nw.gradient(x_batch, t_batch)

# 各重みの絶対誤差の平均値を出す
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + " : " + str(diff))
    
    


IndexError: arrays used as indices must be of integer (or boolean) type